In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.text_data_utils.bert_embeddings import BertFeatureExtractor

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import pandas as pd
import joblib

In [4]:
!ls ../data/text_data/

'JDB_Face_metadata (1).json'   cleaned_DFDB_Face_metadata.csv


In [6]:
jdb_fp = '/home/ginger/code/gderiddershanghai/Diffusion-Face/data/text_data/JDB_Face_metadata (1).json'
df = pd.read_json(jdb_fp)

In [10]:
df.head()

,img_path,prompt,Task1,Task2,Task3,ori_prompt,url
0,84174671-387f-4927-9cd3-df194ac4bf79.jpg,"character art, agent of chaos, cinematic light...","{'Style': ['cinematic lighting', 'highly detai...",{'Caption': 'A highly detailed digital illustr...,{'Style Relevant Questions and Answers': [{'Qu...,"character art, agent of chaos, cinematic light...",https://cdn.discordapp.com/attachments/9954312...
1,2bb2dbb7-8e63-4cef-ac0a-1590cab19ed2.jpg,"Post Malone as Yoda, super realistic, star war...","{'Style': ['super realistic'], 'Content': ['Po...",{'Caption': 'Post Malone takes on the role of ...,{'Style Relevant Questions and Answers': [{'Qu...,"Post Malone as Yoda, super realistic, star war...",https://cdn.discordapp.com/attachments/9954312...
2,2864a9cb-82ce-4501-8306-c9f478533bf7.jpg,"Jim Lee style, James Tyler Williams Lincoln Pe...","{'Style': ['Jim Lee style', 'Harlem Renaissanc...",{'Caption': 'In this close-up image rendered i...,{'Style Relevant Questions and Answers': [{'Qu...,"Jim Lee style, James Tyler Williams Lincoln Pe...",https://cdn.discordapp.com/attachments/9954312...
3,513c2b14-6ba7-4cee-b3eb-5d64314c7ffe.jpg,weird Al yankovic as a poodle,"{'Style': ['weird', 'poodle'], 'Content': ['Al...",{'Caption': 'Weird Al Yankovic transformed int...,{'Style Relevant Questions and Answers': [{'Qu...,weird Al yankovic as a poodle,https://cdn.discordapp.com/attachments/9954315...
4,2e59debf-5727-4c9c-bc42-3e2b2559b8bb.jpg,hindu,"{'Style': ['Hindu art', 'Indian style', 'Relig...",{'Caption': 'A depiction of Hindu deities in a...,{'Style Relevant Questions and Answers': [{'Qu...,hindu --v 4,https://cdn.discordapp.com/attachments/9954315...


In [12]:
extractor = BertFeatureExtractor()
# new_embeddings = extractor.transform(df, text_column=text_column)

# #load model
# model = joblib.load(model_path)
# new_predictions_prob = model.predict_proba(new_embeddings)
# new_predictions = np.where(new_predictions_prob[:, 1] > threshold, 1, 0)

# df['nsfw_prediction'] = new_predictions
# df.to_csv(save_path, index=False)


/home/ginger/.pyenv/versions/3.10.6/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to

In [13]:
extractor = BertFeatureExtractor()
new_embeddings = extractor.transform(df, text_column='prompt')

  0%|          | 0/61983 [00:00<?, ?it/s]

In [17]:
df['nsfw'] = np.nan
df['contains_person'] = np.nan

In [19]:
df = df[['img_path', 'prompt', 'nsfw', 'contains_person']]

In [21]:
model_path = '/home/ginger/code/gderiddershanghai/Diffusion-Face/weights/svm_model_weights_nsfw.pkl'
model = joblib.load(model_path)
new_predictions_prob = model.predict_proba(new_embeddings)


NameError: name 'threshold' is not defined

In [23]:
threshold = 0.85
new_predictions = np.where(new_predictions_prob[:, 1] > threshold, 1, 0)

In [26]:
save_path = '/home/ginger/code/gderiddershanghai/Diffusion-Face/data/text_data/JDB_Face.csv'
df['nsfw'] = new_predictions
df.to_csv(save_path, index=False)

,img_path,prompt,nsfw,contains_person,nsfw_prediction
0,84174671-387f-4927-9cd3-df194ac4bf79.jpg,"character art, agent of chaos, cinematic light...",0,NaN,0
1,2bb2dbb7-8e63-4cef-ac0a-1590cab19ed2.jpg,"Post Malone as Yoda, super realistic, star war...",0,NaN,0
2,2864a9cb-82ce-4501-8306-c9f478533bf7.jpg,"Jim Lee style, James Tyler Williams Lincoln Pe...",0,NaN,0
3,513c2b14-6ba7-4cee-b3eb-5d64314c7ffe.jpg,weird Al yankovic as a poodle,0,NaN,0
4,2e59debf-5727-4c9c-bc42-3e2b2559b8bb.jpg,hindu,0,NaN,0


In [45]:
first_50 = [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 
            0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
            0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 
            0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 
            0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 
            0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 
            1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 
            1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
            0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]
first_400_vals = np.array([first_50[i:i+10] for i in range(0, len(first_50), 10)])

In [47]:
first_400_vals[0] = [0, 1, 0, 0, 0, 0, 1, 0, 1, 0]

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0])

In [61]:
df = df.sort_values(by='img_path', ascending=False).reset_index(drop=True)

In [59]:
np.ones(10)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [69]:
labels[0] = [0, 0, 0, 0, 1, 0, 1, 1, 0, 0]
labels[1] = [0, 1, 1, 1, 1, 0, 1, 1, 1, 0]
labels[2] = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
labels[3] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels[4] = [0, 0, 0, 0, 0, 0, 1, 1, 0, 1]

In [84]:
labels[5] = [0, 0, 0, 0, 1, 0, 1, 1, 0, 1] # labels 50-59
labels[6] = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0] # labels 60-69
labels[7] = [0, 0, 0, 0, 0, 0, 1, 1, 0, 0] # labels 70-79
labels[8] = [0, 1, 1, 1, 1, 0, 1, 1, 0, 0] # labels 80-89
labels[9] = [0, 0, 0, 1, 1, 0, 1, 0, 0, 0] # labels 90-99
labels[10] = [1, 1, 0, 0, 0, 0, 0, 1, 0, 0] # labels 100-109
labels[11] = [1, 0, 0, 0, 0, 0, 0, 0, 1, 0] # labels 110-119
labels[12] = [1, 0, 1, 0, 1, 0, 0, 0, 1, 1] # labels 120-129
labels[13] = [1, 0, 0, 1, 0, 0, 1, 0, 0, 0] # labels 130-139
labels[14] = [0, 0, 1, 0, 0, 1, 1, 1, 0, 0] # labels 140-149
labels[15] = [0, 1, 0, 1, 0, 1, 1, 0, 1, 0] # labels 150-159
labels[16] = [0, 0, 1, 0, 0, 0, 0, 0, 1, 1] # labels 160-169
labels[17] = [1, 0, 1, 1, 0, 0, 0, 1, 1, 0] # labels 170-179
labels[18] = [0, 1, 1, 1, 0, 1, 0, 0, 1, 0] # labels 180-189
labels[19] = [1, 0, 0, 1, 0, 1, 0, 0, 1, 1] # labels 190-199
labels[20] = [1, 0, 0, 0, 0, 0, 1, 0, 0, 0] # labels 200-209
labels[21] = [0, 0, 0, 0, 0, 0, 1, 1, 0, 0] # labels 210-219
labels[22] = [0, 0, 1, 1, 0, 0, 1, 0, 1, 0] # labels 220-229
labels[23] = [1, 0, 0, 0, 0, 0, 0, 0, 1, 0] # labels 230-239
labels[24] = [0, 0, 0, 1, 0, 1, 0, 0, 1, 1] # labels 240-249
labels[25] = [0, 1, 0, 0, 1, 1, 0, 1, 1, 0] # labels 250-259
labels[26] = [0, 0, 1, 1, 1, 0, 0, 1, 1, 0] # labels 260-269
labels[27] = [0, 1, 1, 0, 1, 0, 1, 1, 0, 0] # labels 270-279
labels[28] = [0, 0, 0, 1, 0, 1, 0, 0, 1, 1] # labels 280-289
labels[29] = [1, 0, 0, 1, 0, 1, 0, 1, 0, 0] # labels 290-299
labels[30] = [1, 0, 1, 0, 1, 1, 1, 0, 0, 0] # labels 300-309
labels[31] = [0, 1, 0, 0, 1, 0, 1, 1, 0, 0] # labels 310-319
labels[32] = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0] # labels 320-329
labels[33] = [0, 0, 1, 1, 0, 0, 0, 1, 0, 0] # labels 330-339
labels[34] = [1, 1, 0, 0, 1, 0, 1, 1, 0, 0] # labels 340-349
labels[35] = [0, 1, 0, 0, 1, 0, 0, 1, 1, 0] # labels 350-359
labels[36] = [1, 0, 0, 1, 0, 1, 0, 1, 1, 1] # labels 360-369
labels[37] = [1, 0, 0, 0, 1, 0, 1, 0, 1, 0] # labels 370-379
labels[38] = [1, 1, 0, 0, 1, 1, 1, 0, 0, 0] # labels 380-389
labels[39] = [1, 0, 1, 0, 0, 0, 1, 0, 0, 1] # labels 390-399


In [87]:
for row in df.head(400).itertuples():
    idx = row.Index
        # first_50[idx]
        # print(idx, row.prompt)
    df.at[idx,'contains_person'] =labels.flatten()[idx]

In [100]:
extractor = BertFeatureExtractor()
new_embeddings = extractor.transform(df, text_column='prompt')

/home/ginger/.pyenv/versions/3.10.6/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to

  0%|          | 0/61983 [00:00<?, ?it/s]

In [200]:
jdb_train = df.iloc[:200, :]
jdb_test = df.iloc[200:, :]

y_train = jdb_train['contains_person'].values
X_train = new_embeddings[:200]  
X_test = new_embeddings[200:]  

In [201]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [202]:
best_model

GradientBoostingClassifier(learning_rate=0.01, max_depth=7, n_estimators=50,
                           random_state=42, subsample=0.8)

In [203]:
X_train.shape, X_val.shape

((160, 768), (40, 768))

In [204]:
best_params

{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}

In [205]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, precision_recall_curve, auc
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [206]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
from sklearn.metrics import recall_score

# Dictionary to store the recall for each model
model_recalls = {}

# Iterate over the models
for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
    grid_search.fit(X_train_scaled, y_train)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Make predictions on the validation set
    y_val_pred_proba = best_model.predict_proba(X_val_scaled)
    threshold = 0.5
    y_val_pred = np.where(y_val_pred_proba[:, 1] > threshold, 1, 0)

    # Evaluate recall for class 1
    recall = recall_score(y_val, y_val_pred)
    model_recalls[model_name] = recall

# Find the model with the highest recall for class 1
best_model_name = max(model_recalls, key=model_recalls.get)
best_recall = model_recalls[best_model_name]

# Print the best model and recall
print(f"\nBest model for predicting the most 1s: {best_model_name}")
print(f"Recall for class 1: {best_recall}")


Training and evaluating SVM...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Training and evaluating Logistic Regression...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Training and evaluating Random Forest...
Fitting 3 folds for each of 81 candidates, totalling 243 fits
[CV] END .....................C=0.01, gamma=0.01, kernel=rbf; total time=   0.1s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.2s
[CV] END .......................C=10, gamma=auto, kernel=rbf; total time=   0.3s
[CV] END ................................C=0.1, solver=lbfgs; total time=   0.0s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_split=10, n_estimators=50; total time=   0.2s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=10, min_samples_split=10, 

In [207]:
best_recall,best_model_name 

(0.35714285714285715, 'Naive Bayes')

[CV] END learning_rate=0.01, max_depth=7, min_samples_split=2, n_estimators=100, subsample=0.8; total time=   3.0s
[CV] END learning_rate=0.01, max_depth=7, min_samples_split=5, n_estimators=100, subsample=0.8; total time=   2.6s
[CV] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=50, subsample=1.0; total time=   1.1s
[CV] END learning_rate=0.1, max_depth=3, min_samples_split=2, n_estimators=200, subsample=1.0; total time=   4.0s
[CV] END learning_rate=0.1, max_depth=5, min_samples_split=2, n_estimators=200, subsample=0.8; total time=   4.8s
[CV] END learning_rate=0.1, max_depth=7, min_samples_split=2, n_estimators=100, subsample=1.0; total time=   2.5s
[CV] END learning_rate=0.1, max_depth=7, min_samples_split=5, n_estimators=100, subsample=0.8; total time=   3.1s
[CV] END learning_rate=0.5, max_depth=3, min_samples_split=2, n_estimators=100, subsample=1.0; total time=   1.5s
[CV] END learning_rate=0.5, max_depth=3, min_samples_split=5, n_estimators=50, subsampl

In [104]:

threshold = 0.25
y_val_pred = np.where(y_val_pred_proba[:, 1] > threshold, 1, 0)

# Evaluate the model
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print(classification_report(y_val, y_val_pred))

Validation Accuracy: 0.375
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        50
         1.0       0.38      1.00      0.55        30

    accuracy                           0.38        80
   macro avg       0.19      0.50      0.27        80
weighted avg       0.14      0.38      0.20        80



/home/ginger/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ginger/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ginger/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [113]:
y_val_pred.shape

(80,)

In [178]:
for threshold in np.arange(0,1,0.05):
   
    y_val_pred = np.where(y_val_pred_proba[:, 1] > threshold, 1, 0)
    
    # Evaluate the model
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print("Validation Accuracy:", val_accuracy)
    # print(classification_report(y_val, y_val_pred))


Validation Accuracy: 0.33
Validation Accuracy: 0.33
Validation Accuracy: 0.33
Validation Accuracy: 0.33
Validation Accuracy: 0.33
Validation Accuracy: 0.33
Validation Accuracy: 0.33
Validation Accuracy: 0.33
Validation Accuracy: 0.33
Validation Accuracy: 0.67
Validation Accuracy: 0.67
Validation Accuracy: 0.67
Validation Accuracy: 0.67
Validation Accuracy: 0.67
Validation Accuracy: 0.67
Validation Accuracy: 0.67
Validation Accuracy: 0.67
Validation Accuracy: 0.67
Validation Accuracy: 0.67
Validation Accuracy: 0.67


In [124]:
threshold = 0.5
y_val_pred = np.where(y_val_pred_proba[:, 1] > threshold, 1, 0)

# Evaluate the model
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print(classification_report(y_val, y_val_pred))

Validation Accuracy: 0.625
              precision    recall  f1-score   support

         0.0       0.62      1.00      0.77        50
         1.0       0.00      0.00      0.00        30

    accuracy                           0.62        80
   macro avg       0.31      0.50      0.38        80
weighted avg       0.39      0.62      0.48        80



/home/ginger/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ginger/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ginger/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [ ]:
# Now you can predict on the test set (unlabeled data) and create labels
y_test_pred_proba = svm_model.predict_proba(X_test)
y_test_pred = np.where(y_test_pred_proba[:, 1] > threshold, 1, 0)

# You can now assign the predicted labels to your test set
jdb_test['contains_person_pred'] = y_test_pred